In [0]:
import os
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import IntegerLookup
from tensorflow.keras.layers import Normalization
from tensorflow.keras.layers import StringLookup
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.metrics import classification_report
from collections import Counter

In [0]:
filepath2021="dbfs:/FileStore/output/resnet50v2_train_output.csv"
df_img_2021 = spark.read.format("csv").option("mode", "DROPMALFORMED").option("header", "true").load(filepath2021)

filepath2022="dbfs:/FileStore/output/resnet50v2_test_output.csv"
df_img_2022 = spark.read.format("csv").option("mode", "DROPMALFORMED").option("header", "true").load(filepath2022)

In [0]:
df_img_2021 = df_img_2021.toPandas()
df_img_2022 = df_img_2022.toPandas()

In [0]:
temp = df_img_2021['pred'].apply(lambda x: x.split(','))
temp2022 = df_img_2022['pred'].apply(lambda x: x.split(','))

In [0]:
df_img_2021['pred_cat'] = temp.apply(lambda x: x[1].replace(' ', '').replace("'", ''))
df_img_2022['pred_cat'] = temp2022.apply(lambda x: x[1].replace(' ', '').replace("'", ''))

In [0]:
container ="data1"
data_path = f"abfss://{container}@capstone2023cuhk.dfs.core.windows.net/"
df = spark.read.csv(os.path.join(data_path, "AML-data", "Actual_Data_2021-2022", "ITEM_MATCHING_2021_BOOKS.csv"), header=True)
df2 = spark.read.csv(os.path.join(data_path, "AML-data", "Actual_Data_2021-2022", "ITEM_MATCHING_2022_BOOKS.csv"), header=True)
df = df.toPandas()
df2 = df2.toPandas()
df = df.loc[df['PRICE']!='0']
df = df.loc[df['QTY_SALES']!='0']
df2 = df2.loc[df2['PRICE']!='0']
df2 = df2.loc[df2['QTY_SALES']!='0']
# df.loc[df['QTY_SALES']=='MBI       ']
df = df.drop(index=[583])

In [0]:
df['PRODUCT_ID'] = df['PRODUCT_ID'].str.strip()
df2['PRODUCT_ID'] = df2['PRODUCT_ID'].str.strip()

In [0]:
df = df.merge(df_img_2021, on="PRODUCT_ID")
df2 = df2.merge(df_img_2022, on="PRODUCT_ID")

In [0]:
len(df),len(df2)

Out[9]: (277, 284)

In [0]:
VENDORList = set(df['VENDOR'].str.strip().unique()) & set(df2['VENDOR'].str.strip().unique())
df = df[df['VENDOR'].str.strip().isin(VENDORList)]
df2 = df2[df2['VENDOR'].str.strip().isin(VENDORList)]

In [0]:
BRANDList = set(df['BRAND'].str.strip().unique()) & set(df2['BRAND'].str.strip().unique())
df = df[df['BRAND'].str.strip().isin(BRANDList)]
df2 = df2[df2['BRAND'].str.strip().isin(BRANDList)]

In [0]:
PRD_CATEGORYList = set(df['PRD_CATEGORY'].str.strip().unique()) & set(df2['PRD_CATEGORY'].str.strip().unique())
df = df[df['PRD_CATEGORY'].str.strip().isin(PRD_CATEGORYList)]
df2 = df2[df2['PRD_CATEGORY'].str.strip().isin(PRD_CATEGORYList)]

In [0]:
PRD_ORIGINList = set(df['PRD_ORIGIN'].str.strip().unique()) & set(df2['PRD_ORIGIN'].str.strip().unique())
df = df[df['PRD_ORIGIN'].str.strip().isin(PRD_ORIGINList)]
df2 = df2[df2['PRD_ORIGIN'].str.strip().isin(PRD_ORIGINList)]

In [0]:
PUBLISHERList = set(df['PUBLISHER'].str.strip().unique()) & set(df2['PUBLISHER'].str.strip().unique())
df = df[df['PUBLISHER'].str.strip().isin(PUBLISHERList)]
df2 = df2[df2['PUBLISHER'].str.strip().isin(PUBLISHERList)]

In [0]:
PRECATList = set(df['pred_cat'].str.strip().unique()) & set(df2['pred_cat'].str.strip().unique())
df = df[df['pred_cat'].str.strip().isin(PRECATList)]
df2 = df2[df2['pred_cat'].str.strip().isin(PRECATList)]

In [0]:
df2 = df2.loc[df2['BRAND']!='S-ZONE  ']
df2 = df2.loc[df2['VENDOR']!='HINKLER   ']
df2 = df2.loc[df2['VENDOR']!='66-BOOKS  ']
df2 = df2.loc[df2['VENDOR']!='CDP       ']
df2 = df2.loc[df2['PUBLISHER']!='CDP                      ']
df2 = df2.loc[df2['PUBLISHER']!='CAMPBELL                 ']
df2 = df2.loc[df2['PRD_CATEGORY']!='ART  ']
df2 = df2.loc[df2['BRAND']!='CAMPBELL']
df2 = df2.loc[df2['BRAND']!='CDP     ']
df = df.loc[df['VENDOR']!='RIVERSIDE ']
df = df.loc[df['PRD_CATEGORY']!='HIS  ']

In [0]:
len(df),len(df2)

Out[17]: (96, 134)

In [0]:
def categorizeSales(threshold, n):
    return 1.0 if n >= threshold else 0.0

In [0]:
df3 = df[['COST','PRICE', 'QTY_SALES','VENDOR','BRAND','PRD_CATEGORY','PRD_ORIGIN','PUBLISHER','pred_cat']]
df3

Out[19]:

,COST,PRICE,QTY_SALES,VENDOR,BRAND,PRD_CATEGORY,PRD_ORIGIN,PUBLISHER,pred_cat
7,0.00,119.90,1,HC,HARPER,F,UK,HC,pencil_box
8,0.00,119.90,1,HC,HARPER,F,UK,HC,comic_book
9,0.00,119.90,1,HC,HARPER,F,UK,HC,pencil_box
10,0.00,105.00,2,HC,HARPER,F,UK,HC,packet
11,107.07,389.70,17,SL,SL,G/S,USA,SL,toyshop
...,...,...,...,...,...,...,...,...,...
169,21.95,79.90,10,AOL,L-BROWN,F,USA,L-BROWN,comic_book
170,21.95,79.90,7,SL,SL,F,USA,SL,web_site
171,35.69,129.90,16,SL,SL,COMIC,USA,SL,envelope
172,24.70,89.90,9,SL,SL,F,USA,SL,great_white_shark


In [0]:
df3['COST'] = pd.to_numeric(df['COST'], errors='coerce')
df3 = df3.dropna(subset=['COST'])
df3['COST'] = df3['COST'].astype(float)
df3['PRICE'] = df3['PRICE'].astype(float)
df3['QTY_SALES'] = df3['QTY_SALES'].astype(float)
df3["QTY_SALES"] = minmax_scale(df3["QTY_SALES"], feature_range=(0, 1), axis=0, copy=True)
df3 = pd.get_dummies(df3, prefix="cat", 
                            columns=["VENDOR","BRAND","PRD_CATEGORY","PRD_ORIGIN","PUBLISHER","pred_cat"], 
                            drop_first=False)

<command-2592544856645817>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 df3['COST'] = pd.to_numeric(df['COST'], errors='coerce')

In [0]:
#define trainging data set and testing data set
training_data, testing_data = train_test_split(df3, test_size=0.2, random_state=25)

In [0]:
threshold = 0.3
x_data = training_data.drop(labels=['QTY_SALES'],axis=1)
y_data = training_data['QTY_SALES'].apply(lambda x: categorizeSales(threshold, x))
x_data_test = testing_data.drop(labels=['QTY_SALES'],axis=1)
y_data_test = testing_data['QTY_SALES'].apply(lambda x: categorizeSales(threshold, x))

In [0]:
x_data

Out[23]:

,COST,PRICE,cat_AOL,cat_HC,cat_HC-USA,cat_MML,cat_MPS,cat_PE-USA,cat_PI,cat_RH,cat_S&S,cat_SL,cat_TBS,cat_ANDREWS,cat_BBY,cat_DK(USA),cat_EGM,cat_HARPER,cat_HC-USA,cat_L-BROWN,cat_MML,cat_PENGUIN,cat_PI,cat_PUFFIN,cat_RH,cat_S&S,cat_SL,cat_SL-UK,cat_UB,cat_B.BK,cat_C-B,cat_COMIC,cat_F,cat_G/S,cat_GB,cat_L-T-F,cat_LR-1,cat_LR-2,cat_LR-3,cat_N/F,cat_PIC,cat_REF,cat_SOUND,cat_UK,cat_USA,cat_ANDREWS,cat_BLOOMSBURY,cat_DK(USA),cat_EGM,cat_HC,cat_HC-USA,cat_L-BROWN,cat_MML-C,cat_MML-USA,cat_PENGUIN,cat_PI,cat_PUFFIN,cat_RH,cat_S&S,cat_SL,cat_SL-UK,cat_UB,cat_binder,cat_book_jacket,cat_comic_book,cat_envelope,cat_great_white_shark,cat_menu,cat_packet,cat_pencil_box,cat_rubber_eraser,cat_slot,cat_toyshop,cat_web_site
25,52.89,269.5,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
92,16.46,69.9,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
59,30.32,119.9,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
80,55.26,220.0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
104,29.79,115.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,11.76,59.9,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
28,30.19,109.9,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
168,46.29,119.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
110,11.76,59.9,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [0]:
model = keras.models.Sequential()
model.add(layers.Dense(512, activation='relu',input_dim=x_data.shape[1]))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(loss=keras.losses.BinaryCrossentropy(), optimizer='adam', metrics=['accuracy'])

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type) Output Shape Param # 
=================================================================
 dense (Dense) (None, 512) 38400 
 
 dropout (Dropout) (None, 512) 0 
 
 dense_1 (Dense) (None, 256) 131328 
 
 dropout_1 (Dropout) (None, 256) 0 
 
 dense_2 (Dense) (None, 1) 257 
 
=================================================================
Total params: 169,985
Trainable params: 169,985
Non-trainable params: 0
_________________________________________________________________

In [0]:
len(x_data),len(y_data)

Out[26]: (76, 76)

In [0]:
model.fit(x=x_data, y=y_data, epochs=100)

Epoch 1/100
1/3 [=========>....................] - ETA: 1s - loss: 2.9229 - accuracy: 0.59383/3 [==============================] - 1s 121ms/step - loss: 1.5298 - accuracy: 0.8158
Epoch 2/100
1/3 [=========>....................] - ETA: 0s - loss: 2.0402 - accuracy: 0.93753/3 [==============================] - 0s 4ms/step - loss: 1.6041 - accuracy: 0.9605
Epoch 3/100
1/3 [=========>....................] - ETA: 0s - loss: 1.6296 - accuracy: 0.96883/3 [==============================] - 0s 5ms/step - loss: 1.8568 - accuracy: 0.9605
Epoch 4/100
1/3 [=========>....................] - ETA: 0s - loss: 1.2435 - accuracy: 0.96883/3 [==============================] - 0s 5ms/step - loss: 1.6765 - accuracy: 0.9605
Epoch 5/100
1/3 [=========>....................] - ETA: 0s - loss: 1.0166 - accuracy: 0.96883/3 [==============================] - 0s 9ms/step - loss: 1.1980 - accuracy: 0.9605
Epoch 6/100
1/3 [=========>....................] - ETA: 0s - loss: 0.9144 - accuracy: 0.96883/3 [==============================] - 0s 5ms/step - loss: 0.8957 - accuracy: 0.9605
Epoch 7/100
1/3 [=========>....................] - ETA: 0s - loss: 0.3627 - accuracy: 0.84383/3 [==============================] - 0s 9ms/step - loss: 0.7406 - accuracy: 0.8553
Epoch 8/100
1/3 [=========>....................] - ETA: 0s - loss: 0.8088 - accuracy: 0.81253/3 [==============================] - 0s 5ms/step - loss: 0.8618 - accuracy: 0.7895
Epoch 9/100
1/3 [=========>....................] - ETA: 0s - loss: 0.7175 - accuracy: 0.90623/3 [==============================] - 0s 5ms/step - loss: 0.5308 - accuracy: 0.9211
Epoch 10/100
1/3 [=========>....................] - ETA: 0s - loss: 0.3333 - accuracy: 0.96883/3 [==============================] - 0s 4ms/step - loss: 0.5387 - accuracy: 0.9605
Epoch 11/100
1/3 [=========>....................] - ETA: 0s - loss: 0.0125 - accuracy: 1.00003/3 [==============================] - 0s 4ms/step - loss: 0.5756 - accuracy: 0.9474
Epoch 12/100
1/3 [=========>....................] - ETA: 0s - loss: 0.0143 - accuracy: 1.00003/3 [==============================] - 0s 4ms/step - loss: 0.5731 - accuracy: 0.9605
Epoch 13/100
1/3 [=========>....................] - ETA: 0s - loss: 0.0589 - accuracy: 0.96883/3 [==============================] - 0s 4ms/step - loss: 0.2017 - accuracy: 0.9605
Epoch 14/100
1/3 [=========>....................] - ETA: 0s - loss: 1.1293 - accuracy: 0.78123/3 [==============================] - 0s 8ms/step - loss: 0.6211 - accuracy: 0.8816
Epoch 15/100
1/3 [=========>....................] - ETA: 0s - loss: 0.8709 - accuracy: 0.84383/3 [==============================] - 0s 4ms/step - loss: 0.5434 - accuracy: 0.8947
Epoch 16/100
1/3 [=========>....................] - ETA: 0s - loss: 0.1548 - accuracy: 0.9688

In [0]:
len(x_data_test) , len(y_data_test)

Out[28]: (20, 20)

In [0]:
score, acc = model.evaluate(x_data_test, y_data_test)
print('Test score:', score)
print('Test accuracy:', acc)

1/1 [==============================] - ETA: 0s - loss: 0.0110 - accuracy: 1.00001/1 [==============================] - 0s 169ms/step - loss: 0.0110 - accuracy: 1.0000
Test score: 0.010982653126120567
Test accuracy: 1.0

In [0]:
df4 = df2[['COST','PRICE','VENDOR','BRAND','PRD_CATEGORY','PRD_ORIGIN','PUBLISHER','QTY_SALES','pred_cat']]
df4['COST'] = pd.to_numeric(df2['COST'], errors='coerce')
df4 = df4.dropna(subset=['COST'])
df4['COST'] = df4['COST'].astype(float)
df4['PRICE'] = df4['PRICE'].astype(float)
df4 = pd.get_dummies(df4, prefix="cat", columns=["VENDOR","BRAND","PRD_CATEGORY","PRD_ORIGIN","PUBLISHER","pred_cat"], drop_first=False)
df4['QTY_SALES'] = minmax_scale(df4["QTY_SALES"], feature_range=(0, 1), axis=0, copy=True)

<command-2592544856645827>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 df4['COST'] = pd.to_numeric(df2['COST'], errors='coerce')

In [0]:
y = df4['QTY_SALES'].apply(lambda x: categorizeSales(threshold, x))
x = df4.loc[:, ~df4.columns.isin(['QTY_SALES'])]

In [0]:
score, acc = model.evaluate(x, y)
print('Test score:', score)
print('Test accuracy:', acc)

1/5 [=====>........................] - ETA: 0s - loss: 0.2940 - accuracy: 0.96885/5 [==============================] - 0s 2ms/step - loss: 0.2960 - accuracy: 0.9552
Test score: 0.2960050702095032
Test accuracy: 0.9552238583564758

In [0]:
y_pred = np.round(model.predict(x))
y_pred = y_pred.astype("float64")

In [0]:
target_names = ['class 0', 'class 1']
print(classification_report(y, y_pred, target_names=target_names))

/databricks/python/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
 _warn_prf(average, modifier, msg_start, len(result))
/databricks/python/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
 _warn_prf(average, modifier, msg_start, len(result))
/databricks/python/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
 _warn_prf(average, modifier, msg_start, len(result))
 precision recall f1-score support

 class 0 0.96 1.00 0.98 128
 class 1 0.00 0.00 0.00 6

 accuracy 0.96 134
 macro avg 0.48 0.50 0.49 134
weighted avg 0.91 0.96 0.93 134

In [0]:
def runModel(df_train, df_test, model ,threshold,accuracy):
    x_train = df_train.drop(labels=['QTY_SALES'],axis=1)
    y_train = df_train["QTY_SALES"].apply(lambda x: categorizeSales(threshold, x))

    x_test = df_test.drop(labels=['QTY_SALES'],axis=1)
    y_test = df_test["QTY_SALES"].apply(lambda x: categorizeSales(threshold, x))

    model.fit(x=x_train, y=y_train, epochs=100,verbose=0)

    score, acc = model.evaluate(x_test, y_test)
    print("Threshold: {}".format(str(threshold)))
    print('Test score:', score)
    print('Test accuracy:', acc)
    target_names = ['class 0', 'class 1']

    y_train_pred = np.round(model.predict(x_train))
    y_train_pred = y_train_pred.astype("float64")
    print(classification_report(y_train, y_train_pred, target_names=target_names))

    y_pred = np.round(model.predict(x_test))
    y_pred = y_pred.astype("float64")
    print(classification_report(y_test, y_pred, target_names=target_names))

    accuracy= accuracy.append(acc)

In [0]:
accuracy = []
for t in range(1, 10):
    threshold = t/100
    runModel(df3, df4,model,threshold,accuracy)

INFO:tensorflow:Assets written to: /tmp/tmpgz6r_4nf/model/data/model/assets
1/5 [=====>........................] - ETA: 0s - loss: 0.3839 - accuracy: 0.90625/5 [==============================] - 0s 2ms/step - loss: 0.4181 - accuracy: 0.8881
Threshold: 0.01
Test score: 0.41809219121932983
Test accuracy: 0.888059675693512
 precision recall f1-score support

 class 0 0.80 0.36 0.50 11
 class 1 0.92 0.99 0.95 85

 accuracy 0.92 96
 macro avg 0.86 0.68 0.73 96
weighted avg 0.91 0.92 0.90 96

 precision recall f1-score support

 class 0 1.00 0.06 0.12 16
 class 1 0.89 1.00 0.94 118

 accuracy 0.89 134
 macro avg 0.94 0.53 0.53 134
weighted avg 0.90 0.89 0.84 134

INFO:tensorflow:Assets written to: /tmp/tmpl6w250y4/model/data/model/assets
1/5 [=====>........................] - ETA: 0s - loss: 0.6808 - accuracy: 0.62505/5 [==============================] - 0s 2ms/step - loss: 0.6190 - accuracy: 0.6716
Threshold: 0.02
Test score: 0.6190083026885986
Test accuracy: 0.6716417670249939
 precision recall f1-score support

 class 0 0.75 0.64 0.69 28
 class 1 0.86 0.91 0.89 68

 accuracy 0.83 96
 macro avg 0.81 0.78 0.79 96
weighted avg 0.83 0.83 0.83 96

 precision recall f1-score support

 class 0 0.47 0.16 0.24 43
 class 1 0.70 0.91 0.79 91

 accuracy 0.67 134
 macro avg 0.58 0.54 0.52 134
weighted avg 0.62 0.67 0.61 134

INFO:tensorflow:Assets written to: /tmp/tmpgyxqe9ss/model/data/model/assets
1/5 [=====>........................] - ETA: 0s - loss: 0.9137 - accuracy: 0.50005/5 [==============================] - 0s 2ms/step - loss: 0.7146 - accuracy: 0.6343
Threshold: 0.03
Test score: 0.7146362662315369
Test accuracy: 0.6343283653259277
 precision recall f1-score support

 class 0 0.97 0.69 0.80 51
 class 1 0.73 0.98 0.84 45

 accuracy 0.82 96
 macro avg 0.85 0.83 0.82 96
weighted avg 0.86 0.82 0.82 96

 precision recall f1-score support

 class 0 0.62 0.59 0.61 64
 class 1 0.64 0.67 0.66 70

 accuracy 0.63 134
 macro avg 0.63 0.63 0.63 134
weighted avg 0.63 0.63 0.63 134

INFO:tensorflow:Assets written to: /tmp/tmpcfcitv1p/model/data/model/assets
1/5 [=====>........................] - ETA: 0s - loss: 1.5213 - accuracy: 0.50005/5 [==============================] - 0s 2ms/step - loss: 1.0154 - accuracy: 0.6045
Threshold: 0.04
Test score: 1.0153748989105225
Test accuracy: 0.60447758436203
 precision recall f1-score support

 class 0 0.96 0.78 0.86 65
 class 1 0.67 0.94 0.78 31

 accuracy 0.83 96
 macro avg 0.82 0.86 0.82 96
weighted avg 0.87 0.83 0.84 96

 precision recall f1-score support

 class 0 0.67 0.68 0.67 81
 class 1 0.50 0.49 0.50 53

 accuracy 0.60 134
 macro avg 0.59 0.58 0.59 134
weighted avg 0.60 0.60 0.60 134

INFO:tensorflow:Assets written to: /tmp/tmplzj2vu2v/model/data/model/assets
1/5 [=====>........................] - ETA: 0s - loss: 1.1119 - accuracy: 0.56255/5 [==============================] - 0s 2ms/step - loss: 0.9868 - accuracy: 0.6940
Threshold: 0.05
Test score: 0.9867563843727112
Test accuracy: 0.6940298676490784
 precision recall f1-score support

 class 0 0.96 0.93 0.94 72
 class 1 0.81 0.88 0.84 24

 accuracy 0.92 96
 macro avg 0.88 0.90 0.89 96
weighted avg 0.92 0.92 0.92 96

 precision recall f1-score support

 class 0 0.74 0.89 0.81 96
 class 1 0.42 0.21 0.28 38

 accuracy 0.69 134
 macro avg 0.58 0.55 0.54 134
weighted avg 0.65 0.69 0.66 134

INFO:tensorflow:Assets written to: /tmp/tmp8quxbp3g/model/data/model/assets
1/5 [=====>........................] - ETA: 0s - loss: 1.2715 - accuracy: 0.75005/5 [=======================

In [0]:
for t in range(1, 30):
    threshold = t/100
    temp = df3["QTY_SALES"].apply(lambda x: categorizeSales(threshold, x))
    print(threshold)
    print(Counter(temp))

0.01
Counter({1.0: 85, 0.0: 11})
0.02
Counter({1.0: 68, 0.0: 28})
0.03
Counter({0.0: 51, 1.0: 45})
0.04
Counter({0.0: 65, 1.0: 31})
0.05
Counter({0.0: 72, 1.0: 24})
0.06
Counter({0.0: 78, 1.0: 18})
0.07
Counter({0.0: 83, 1.0: 13})
0.08
Counter({0.0: 87, 1.0: 9})
0.09
Counter({0.0: 89, 1.0: 7})
0.1
Counter({0.0: 90, 1.0: 6})
0.11
Counter({0.0: 92, 1.0: 4})
0.12
Counter({0.0: 93, 1.0: 3})
0.13
Counter({0.0: 93, 1.0: 3})
0.14
Counter({0.0: 93, 1.0: 3})
0.15
Counter({0.0: 93, 1.0: 3})
0.16
Counter({0.0: 93, 1.0: 3})
0.17
Counter({0.0: 93, 1.0: 3})
0.18
Counter({0.0: 93, 1.0: 3})
0.19
Counter({0.0: 93, 1.0: 3})
0.2
Counter({0.0: 93, 1.0: 3})
0.21
Counter({0.0: 93, 1.0: 3})
0.22
Counter({0.0: 93, 1.0: 3})
0.23
Counter({0.0: 93, 1.0: 3})
0.24
Counter({0.0: 93, 1.0: 3})
0.25
Counter({0.0: 93, 1.0: 3})
0.26
Counter({0.0: 93, 1.0: 3})
0.27
Counter({0.0: 93, 1.0: 3})
0.28
Counter({0.0: 93, 1.0: 3})
0.29
Counter({0.0: 93, 1.0: 3})

In [0]:
accuracy

Out[127]: [0.888059675693512,
 0.6791045069694519,
 0.641791045665741,
 0.60447758436203,
 0.6865671873092651,
 0.8358209133148193,
 0.8507462739944458,
 0.8805969953536987,
 0.8805969953536987,
 0.9029850959777832,
 0.9253731369972229,
 0.9477611780166626,
 0.9477611780166626,
 0.9477611780166626,
 0.9477611780166626,
 0.9552238583564758,
 0.9552238583564758,
 0.9552238583564758,
 0.9552238583564758,
 0.9552238583564758,
 0.9552238583564758,
 0.9552238583564758,
 0.9552238583564758,
 0.9552238583564758,
 0.9552238583564758,
 0.9552238583564758,
 0.9552238583564758,
 0.9552238583564758,
 0.9552238583564758]